In [1]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

pd.options.display.max_columns = 50

In [2]:
class DataFrameImporter:
    """
    Imports files in a directory and concats to a single dataframe
    """
    
    def __init__(self, path=None):
        self._df = self.search_directory(path)
        
    @property
    def df(self):
        return self._df
    @df.setter
    def df(self, value):
        self._df = value
        
    @staticmethod
    def search_directory(path=None):
        if path is not None:
            building_dfs = []
            for file in os.listdir(path):
                if file.endswith('xlsx'):
                    df = pd.read_excel(os.path.join(path, file), 
                                    na_values='-', 
                                    parse_dates=['Time'], 
                                    date_format='%Y-%m-%d %H:%M:%S')
                    building_dfs.append(df)
            return pd.concat(building_dfs)
        raise ValueError(f'Incorrect path. \tPath: {path}')


In [3]:
class ListSplitter:
    """
    Iterates through a list of integers. If difference between an integer and the next
    is more than 1, it will split the list.
    
    Returns a list of all split lists
    """
    
    @staticmethod
    def split_list_on_increment(lst):
        # get list of indices, if values in list increment by more than 1 split list
        sublists = []
        sublist = []
        for i in range(len(lst)):
            if i == 0 or lst[i] - lst[i-1] <= 1:
                sublist.append(lst[i])
            else:
                sublists.append(sublist)
                sublist = [lst[i]]
        sublists.append(sublist)
        # return all split lists (intervals)
        return sublists      

In [4]:
class DataFrameCleaner:
    """
    Implements basic cleaning functions to a DataFrame
    
    Can use an existing dataframe object, or import from path directory using DataFrame importer
    
    Specify building_no at instantiation as well
    """
    
    def __init__(self, df=None, path=None, building_no=None):
        if (df):
            self._df = df.sort_values()
        elif (path):
            self._df = DataFrameImporter(path).df
        else:
            raise ValueError("'df' or 'path' need to be specified")
        
        self.remap_columns()
        self.adjust_index()
        self.set_dtypes()
        self.add_time_columns()
        self.add_temp_columns()
        self.add_cumsum_columns()
        self.df['building_no'] = building_no
        
    @property
    def df(self):
        return self._df
    @df.setter
    def df(self, value):
        self._df = value
        
    def remap_columns(self):
        columns_map = {'Zone name':'Zone_name',
                    'Time':'Datetime',
                    'Zone temperature (degree Celsius)':'Zone_temp',
                    'Slab temperature (degree Celsius)': 'Slab_temp',
                    'Dew point temperature (degree Celsius)':'Dew_temp',
                    'Outside air temperature (degree Celsius)':'Ambient_temp',
                    'Damper status (%)':'Damper_status',
                    'Damper status':'Damper_status',
                    'Fan status':'Fan_status',
                    'Zone CO2 (ppm)':'Zone_co2',
                    'Louver status':'Louver_status'}
        self.df = self._df.rename(columns=columns_map)
    
    def adjust_index(self):
        self.df = self._df.sort_values(['Zone_name','Datetime']).reset_index(drop=True)

    def set_dtypes(self):
        dtypes_map = {'Zone_name':'object',
                    'Datetime':'datetime64[ns]',
                    'Zone_temp':float,
                    'Slab_temp':float,
                    'Dew_temp':float,
                    'Ambient_temp':float,
                    'Damper_status':float,
                    'Fan_status':'object',
                    'Zone_co2':float,
                    'Louver_status':'object'}
        
        for k,v in dtypes_map.items():
            if k in self.df.columns:
                self.df[k] = self.df[k].astype(v)
                
    def add_time_columns(self):
        # calc time between records
        self.df['Datetime_diff_mins'] = np.nan
        for (_, temp) in self.df.groupby(['Zone_name']):
            self.df.loc[temp.index,'Datetime_diff_mins'] = (temp.Datetime.diff()).dt.total_seconds() // 60

        self.df['Date'] = self.df.Datetime.dt.date
        self.df['Time'] = self.df.Datetime.dt.time
        self.df['Year'] = self.df.Datetime.dt.year
        self.df['Month'] = self.df.Datetime.dt.month
        self.df['Day'] = self.df.Datetime.dt.day
        self.df['DOW'] = self.df.Datetime.dt.dayofweek
        
        season_map = {12:1,1:1,2:1,3:2,4:2,5:2,6:3,7:3,8:3,9:4,10:4,11:4}
        self.df['Season'] = self.df.Month.map(season_map)
        

    def add_temp_columns(self):
        self.df['Zone_temp_diff'] = np.nan
        self.df['Slab_temp_diff'] = np.nan
        self.df['Dew_temp_diff'] = np.nan
        self.df['Ambient_temp_diff'] = np.nan
        
        for (_, temp) in self.df.groupby(['Zone_name']):
            self.df.loc[temp.index,'Zone_temp_diff'] = temp.Zone_temp.diff()
            self.df.loc[temp.index,'Slab_temp_diff'] = temp.Slab_temp.diff()
            self.df.loc[temp.index,'Dew_temp_diff'] = temp.Dew_temp.diff()
            self.df.loc[temp.index,'Ambient_temp_diff'] = temp.Ambient_temp.diff()
        
        
    def add_cumsum_columns(self):     
        # Fan status
        self.df['Cumulative_fan_on_mins'] = np.nan
        # self.df['Cumulative_fan_off_mins'] = np.nan
        self.df['Fan_on_group'] = np.nan
        
        # Damper status (0 to 100)
        if 'Damper_satus' in self.df.columns:
            self.df['Cumulative_damper_open_mins'] = np.nan
            self.df['Damper_open_group'] = np.nan
            
        # Louver = Close/Open
        if 'Louver_status' in self.df.columns:
            self.df['Cumulative_louver_open_mins'] = np.nan
            self.df['Louver_open_group'] = np.nan
        
        
        for (_, temp) in self.df.groupby(['Zone_name']):
            # calc cumulative time fan is on for each interval
            on_indices = ListSplitter.split_list_on_increment(temp[temp.Fan_status=='On'].index)
            for i, intervals in enumerate(on_indices):
                self.df.loc[intervals,'Cumulative_fan_on_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
                self.df.loc[intervals,'Fan_on_group'] = i+1
                
            # # calc cumulative time fan is off for each interval
            # off_indices = split_list_on_increment(temp[temp.Fan_status=='Off'].index)
            # for intervals in off_indices:
            #     self.df.loc[intervals,'Cumulative_fan_off_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
            
            if 'Damper_status' in self.df.columns:
                open_indices = ListSplitter.split_list_on_increment(temp[temp.Damper_status==100].index)
                for i, intervals in enumerate(open_indices):
                    self.df.loc[intervals,'Cumulative_damper_open_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
                    self.df.loc[intervals,'Damper_open_group'] = i+1
            
            if 'Louver_status' in self.df.columns:                  
                open_indices = ListSplitter.split_list_on_increment(temp[temp.Louver_status=='Open'].index)
                for i, intervals in enumerate(open_indices):
                    self.df.loc[intervals,'Cumulative_louver_open_mins'] = ((self.df.loc[intervals,'Datetime'].diff()).dt.total_seconds() // 60).cumsum()
                    self.df.loc[intervals,'Louver_open_group'] = i+1

In [65]:
test = DataFrameCleaner(path=r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\exploratory_data\b3', building_no=3)
# test = DataFrameCleaner(path=r'C:\Users\barac\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\exploratory_data\b3', building_no=3)

#### For all Validation Checks:</br><b>Return True if FAULTY/INVALID</b>

In [66]:
# Temp:
# Faulty if any of the following are true:
    # Sudden drop in temp (typically to 0 or -1)
    # Value is out of range
    # Constant value for 24hrs+                   
class TemperatureValidation:
    """
    Class containing all temperature validation checks, works by specifying an area.
    
    eg area='Zone' will do all temperature validation checks for Zone_temp column
    """
    
    def __init__(self, df):
        self._df = df
        
    @property
    def df(self):
        return self._df
    
    def check_invalid_temperature_range(self, area, min=-5, max=50):
        # Return bool series of values that are not within valid temperature range
        return ~((self.df[f'{area}_temp'] > min) & (self.df[f'{area}_temp'] < max))
    
    def check_suboptimal_temperature_range(self, area, min=20, max=24):
        # Return bool series of values that are not within optimal temperature range
        return ~((self.df[f'{area}_temp'] > min) & (self.df[f'{area}_temp'] < max))
         
    def check_sudden_change(self, area, drop=15):
        # Return bool series of values that have not changed it in temp minimally
        return ~(abs(self.df[f'{area}_temp_diff']) < drop)
    
    def check_for_constant(self, area, hours=24):
        # Create bool series, set values to false
        temp_series = pd.Series([False for _ in range(len(self.df))], index=self.df.index)
        for (_, df) in self.df.groupby(['Zone_name']):
            constant_indices = ListSplitter.split_list_on_increment(df[df[f'{area}_temp_diff']==0].index)
            for indices in constant_indices:
                if len(indices)>1:
                    # total time = most recent time minus oldest time
                    total_time = (df.at[indices[-1],'Datetime'] - df.at[indices[0],'Datetime']).total_seconds() / 3600
                    if total_time >= hours:
                        # if total time is more than 24 hrs update all rows in between to True (invalid)
                        temp_series.loc[indices] = True
        return temp_series

In [67]:
# Zone co2:
    # Faulty if > 800ppm
class C02Validation:
    """
    Class containing C02 validation checks
    """
    
    def __init__(self, df):
        self._df = df
        
    @property
    def df(self):
        return self._df
        
    def check_invalid_ppm(self, max_ppm=800):
        return self.df.Zone_co2 > max_ppm

In [68]:
class OperatingHours:
    """
    Class containing building hour information
    
    Must set building_no when instantiating, can also optionally set amount of hours
    prior to open is the optimum start time for each building. default is 2
    """
    
    def __init__(self, building_no, optimum=2):
        self._df_oh = self.get_operating_hours(building_no, optimum)
        
    @property
    def df_oh(self):
        return self._df_oh
            
    @staticmethod
    def get_operating_hours(building_no, optimum):
        match building_no:
            case 1:
                oh_dict = {'open':[8,8,7,8,8,10,12], 'close':[21,21,21,21,21,17,17]}
            case 2:
                oh_dict = {'open':[8,8,8,8,8,8,8], 'close':[17,17,17,17,17,17,17]}
            case 3:
                oh_dict = {'open':[8,8,7,8,8,10,12], 'close':[18,18,18,18,18,18,18]}
            case _:
                print('Invalid building.\t Value: {building_no}')
                
        oh_dict['open'] = [datetime.time(hour) for hour in oh_dict['open']]
        oh_dict['close'] = [datetime.time(hour) for hour in oh_dict['close']]
        
        df = pd.DataFrame(oh_dict, index=[i for i in range (7)])
        df['optimum'] = (df.open.apply(lambda x: pd.to_datetime(x.strftime('%H:%M:%S'))) - datetime.timedelta(hours=optimum)).dt.time
        
        return df[['optimum','open','close']]
    
# Fan status:
# Faulty if:
    # Fan running anytime outside of operating hours, until optimum start time of building
    # Running when zone temp is within ideal range (20 to 24)
    # Running during occupied hrs when ambient temp < 18 or greater than 23 and zone co2 < 800 ppm and damper is fully open
class FanValidation:
    """
    Class containing fan validations
    
    Instantiate by specying building_no along with dataframe object
    """
    
    def __init__(self, df, building_no=None):
        if not (building_no):
            raise ValueError('building_no needs to be specified')
        self._df = df
        self._df_oh = OperatingHours(building_no).df_oh
        
    @property
    def df(self):
        return self._df
    
    @property
    def df_oh(self):
        return self._df_oh
    
    def check_fan_on_outside_oh(self, open_time='optimum'):
        # Get bool series of fan being on outside of operating hours up to open time (default=optimum)
        temp_series = pd.Series([False for _ in range(len(self.df))], index=self.df.index)
        
        # For each day of the week, check valid open and closing times
        for (_, df) in self.df[self.df.Fan_status=='On'].groupby('DOW'):
            # Get indices of when fan was on during closed hours
            indices = df[(df.Time>self.df_oh.at[df.DOW.unique()[0], 'close']) |\
                         (df.Time<self.df_oh.at[df.DOW.unique()[0], open_time])].index
            # Update bool series
            temp_series.loc[indices] = True
        return temp_series
    
    # def check_fan_on_in_ideal_range(self, min=20, max=24):      
    #     # Get bool series of all zone temps in ideal range
    #     within_ideal_range = (self.df.Zone_temp > min) & (self.df.Zone_temp < max)
    #     valid_bools = (within_ideal_range)&(self.df.Fan_status=='Off')

    #     # return invalid values
    #     return ~(valid_bools)
    
    def check_fan_on_in_ideal_range(self, min=20, max=24):      
        # Get bool series of all zone temps in ideal range
        return (self.df.Zone_temp>min)&(self.df.Zone_temp<max)&(self.df.Fan_status=='Off')
    
    def check_fan_on_occupied_co2(self):
        # Get bool series where fan was on during occupied hours
        fan_on_during_occupied = pd.Series([False for _ in range(len(self.df))], index=self.df.index)
        for (_, df) in self.df[self.df.Fan_status=='On'].groupby('DOW'):
            indices = df[(df.Time<self.df_oh.at[df.DOW.unique()[0], 'close']) &\
                         (df.Time>self.df_oh.at[df.DOW.unique()[0], 'open'])].index
            fan_on_during_occupied.loc[indices] = True
            
        # Get bool series where temps were < 18 or > 23
        not_in_ideal_temperature_range = TemperatureValidation(self.df).check_suboptimal_temperature_range('Ambient', min=18, max=23)
        
        # Get bool series where co2 levels were < 800 ppm
        co2_levels_valid = ~C02Validation(self.df).check_invalid_ppm()
        
        # Get bool series where damper was fully open
        damper_fully_open = (self.df.Damper_status==100)
        
        # Return bool series of all conditions being met or not
        return (fan_on_during_occupied & not_in_ideal_temperature_range & co2_levels_valid & damper_fully_open)

In [69]:
# Dampers and louvers:
# Faulty:
    # Remain open for 24hrs +
    # Continuous oscillation for 6+ hrs (open, then closed, then open, etc)
    # Louver is closed during occupied hours while ambient temp is between 18 and 23 degrees and dew point is below 19 degrees
    # Louver is closed outside operating hours with diff between ambient temp and zone temp greater than or equal to 3 and dew temp is less than 19
class DamperValidation:
    """
    Incomplete
    """
    
    def __init__(self, df, building_no=None):
        if not (building_no):
            raise ValueError('building_no needs to be specified')
        self._df = df
        self._df_oh = OperatingHours(building_no).df_oh
        
    @property
    def df(self):
        return self._df
    
    @property
    def df_oh(self):
        return self._df_oh
    
    def check_damper_open_for_too_long(self, hours=24):
        damper_open_groups = self.df[self.df.Cumulative_damper_open_mins/60 >= hours].Damper_open_group.unique()
        return (self.df.Damper_open_group.isin(damper_open_groups))
    
    def check_damper_oscillation(self, hours=6):
        pass
    
    
class LouverValidation:
    """
    Incomplete
    """
    
    def __init__(self, df, building_no=None):
        if not (building_no):
            raise ValueError('building_no needs to be specified')
        self._df = df
        self._df_oh = OperatingHours(building_no).df_oh
        
    @property
    def df(self):
        return self._df
    
    @property
    def df_oh(self):
        return self._df_oh
        
    def check_louver_open_for_too_long(self, hours=24):
        louver_open_groups = self.df[self.df.Cumulative_louver_open_mins/60 >= hours].Damper_open_group.unique()
        return (self.df.Louver_open_group.isin(louver_open_groups))
    
    def check_louver_oscillation(self, hours=6):
        pass
    
    def check_louver_closed_occupied_valid_temp(self):
        pass
    
    def check_louver_closed_occupied_diff_temps(self):
        pass
    
    

In [70]:
t2 = test.df

In [72]:
valid_fan = FanValidation(t2, 3)
valid_co2 = C02Validation(t2)
valid_temp = TemperatureValidation(t2)


a = valid_fan.check_fan_on_in_ideal_range() # complete
b = valid_fan.check_fan_on_occupied_co2() # complete
c = valid_fan.check_fan_on_outside_oh() # complete

d = valid_co2.check_invalid_ppm() # complete

e = valid_temp.check_for_constant('Zone') # complete
f = valid_temp.check_invalid_temperature_range('Zone') # complete?
h = valid_temp.check_sudden_change('Zone') # complete

i = valid_temp.check_for_constant('Dew') # complete
j = valid_temp.check_invalid_temperature_range('Dew') # complete?
l = valid_temp.check_sudden_change('Dew') # complete

m = valid_temp.check_for_constant('Ambient') # complete
n = valid_temp.check_invalid_temperature_range('Ambient') # complete?
p = valid_temp.check_sudden_change('Ambient') # complete

q = valid_temp.check_for_constant('Slab') # complete
r = valid_temp.check_invalid_temperature_range('Slab') # complete?
t = valid_temp.check_sudden_change('Slab') # complete

# still need to finish off louver and damper validations


In [78]:
# Create a set of all invalid indices
all_inds = [a[a].index, 
            b[b].index, 
            c[c].index, 
            d[d].index, 
            e[e].index, 
            f[f].index, 
            h[h].index, 
            i[i].index, 
            j[j].index, 
            l[l].index, 
            m[m].index, 
            n[n].index, 
            p[p].index, 
            q[q].index, 
            r[r].index, 
            t[t].index]

inds_set = set()

for val in all_inds:
    if isinstance(val, pd.RangeIndex):
        inds_set.update(val.tolist())  # Convert RangeIndex to list before adding to set
    else:
        inds_set.update(val)

In [79]:
# drop all invalid indices + drop any columns where all NaN values
df_valid = t2.drop(inds_set).dropna(how='all', axis=1)

In [81]:
display(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
Index: 250644 entries, 41331 to 661270
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Zone_name                    250644 non-null  object        
 1   Datetime                     250644 non-null  datetime64[ns]
 2   Zone_temp                    250644 non-null  float64       
 3   Slab_temp                    250644 non-null  float64       
 4   Dew_temp                     250644 non-null  float64       
 5   Ambient_temp                 250644 non-null  float64       
 6   Zone_co2                     250644 non-null  float64       
 7   Damper_status                250644 non-null  float64       
 8   Fan_status                   250644 non-null  object        
 9   Datetime_diff_mins           250644 non-null  float64       
 10  Date                         250644 non-null  object        
 11  Time                       

None

In [ ]:



    

                    
         
                

# test = DataFrameCleaner(path=r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\exploratory_data\b1')



       

# class ErrorValidator:
#     def __init__(self, df):
#         self._df = df
#         self._error_df = pd.DataFrame(index=self.df.index)
        
#     def errors_temp(self, area):
#         self._error_df[f'{area}_sudden_drop'] = self.

# Temp:
# Faulty if any of the following are true:
    # Sudden drop in temp (typically to 0 or -1)
    # Value is out of range
    # Constant value for 24hrs+
    
# Zone co2:
    # Faulty if > 800ppm
    
# Dampers and louvers:
# Faulty:
    # Remain open for 24hrs +
    # Continuous oscillation for 6+ hrs (open, then closed, then open, etc)
    # Louver is closed during occupied hours while ambient temp is between 18 and 23 degrees and dew point is below 19 degrees
    # Louver is closed outside operating hours with diff between ambient temp and zone temp greater than or equal to 3 and dew temp is less than 19
    
# Fan status:
# Faulty if:
    # Fan running anytime outside of operating hours, until optimum start time of building
    # Running when zone temp is within ideal range (20 to 24)
    # Running during occupied hrs when ambient temp < 18 or greater than 23 and zone co2 < 800 ppm and damper is fully open
    
    
# Additional info:
# Operating hours:
    # B1:
    # {1:[8,21], 2:[8,21], 3:[7,21], 4:[8,21], 5:[8,21], 6:[10,17], 7:[12,17]}
    
    # B2:
    # {1:[8,17], 2:[8,17], 3:[8,17], 4:[8,17], 5:[8,17], 6:[8,17], 7:[8,17]}
    
    # B3:
    # {1:[8,18], 2:[8,18], 3:[7,18], 4:[8,18], 5:[8,18], 6:[10,18], 7:[12,18]}
    
# Optimum start time:
    # 2 hrs earlier
    
# Louver/Damper status:
    # 0 = closed, 1 = open
    
# Valid temp ranges:
    # Overall: -5 to 50
    # Optimum: 18 to 22



# To do:
    # Add temp diff columns ## DONE
    # Sanity check not same temp for 24+ hrs
    # Make sure values are in valid overall range
    # add bool column for whether temp things have pass/failed
    
    # add cumsum time counts of when damper and louver are open (1) ## DONE
        # if open for 24+ hrs, failed
    # add logic to check if values oscillate for 6+ hours
    # add logic for:
        # Louver is closed during occupied hours while ambient temp is between 18 and 23 degrees and dew point is below 19 degrees
        # Louver is closed outside operating hours with diff between ambient temp and zone temp greater than or equal to 3 and dew temp is less than 19
        
    # check if fan running outside of operating hours, until optimum start time
    # check if fan running when temp is betwen 18 and 23 degrees
    # check if fan running during occupied hrs when ambient temp < 18 or greater than 23 and zone co2 < 800 ppm and damper is fully open
    

# Check boxplots of diff in temps 
# ask rishabh:
    # about louver status, is it open if it is greater than 0, or only when fully open at 1?
    # is optimal temp range 18 to 22, or 18 to 23, or?
    # what is a sudden drop, how many degrees? does it need to end up at 0 and/or -1?


    
        
# 20 to 24 optimal
# 15 degree drop

